In [1]:
import seqeval

import evaluate

metric = evaluate.load("seqeval")

import json
import sys
import numpy as np
import evaluate
import torch
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForTokenClassification

def load_dnrti():
    ret = {}
    for split_name in ['train_json', 'dev_json', 'test_json']:
        data = []
        with open(f'/home/vikrant/Desktop/NER/DNRTI/{split_name}.jsonl', 'r') as reader:
            for line in reader:
                data.append(json.loads(line))
        ret[split_name] = Dataset.from_list(data)
    return DatasetDict(ret)

ds = load_dnrti()

label2id = {
    'I-Org': 0,
    'I-SecTeam': 1,
    'O': 2,
    'B-Exp': 3,
    'I-Purp': 4,
    'B-Purp': 5,
    'I-Features': 6,
    'I-Time': 7,
    'B-SecTeam': 8,
    'B-SamFile': 9,
    'B-Area': 10,
    'I-Area': 11,
    'B-HackOrg': 12,
    'B-Way': 13,
    'B-OffAct': 14,
    'B-Org': 15,
    'I-Exp': 16,
    'I-OffAct': 17,
    'B-Features': 18,
    'B-Time': 19,
    'I-SamFile': 20,
    'I-Way': 21,
    'I-HackOrg': 22
}


checkpoint = "google/electra-base-discriminator"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, add_prefix_space=True)


epochs = 4
batch_size = 16
learning_rate = 1e-4
max_length = 120


id2label = {v: k for k, v in label2id.items()}
label_list = list(label2id.keys()) # ds["train"].features[f"ner_tags"].feature.names


model = AutoModelForTokenClassification.from_pretrained(
    checkpoint, num_labels=len(label2id), id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True,
)
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"

model = model.to(device)

print("Current CUDA Device: [{}] {}".format(torch.cuda.current_device(), torch.cuda.get_device_name(torch.cuda.current_device())))
print("Number of CUDA Devices: {}".format(torch.cuda.device_count()))

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True, padding='longest', max_length=max_length, truncation=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_ds = ds.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

training_args = TrainingArguments(
    output_dir="my_awesome_ds_model",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train_json"],
    eval_dataset=tokenized_ds["test_json"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()





/home/vikrant/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Map:   0%|          | 0/5260 [00:00<?, ? examples/s]

Map:   0%|          | 0/661 [00:00<?, ? examples/s]

Map:   0%|          | 0/663 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.233650,0.731270,0.766866,0.748645,0.925578
2,0.356100,0.187615,0.768059,0.839880,0.802366,0.943279
3,0.356100,0.156362,0.830397,0.865500,0.847585,0.955324
4,0.139600,0.157924,0.842798,0.874466,0.858340,0.958095


TrainOutput(global_step=1316, training_loss=0.20574628545882853, metrics={'train_runtime': 154.8006, 'train_samples_per_second': 135.917, 'train_steps_per_second': 8.501, 'total_flos': 1169361648447360.0, 'train_loss': 0.20574628545882853, 'epoch': 4.0})